In [1]:
def Kullback_Leibler(p, p_ml):
    """
    This function calculate the Kullback Leibler Divergence of p and p_ml.
    :param p:
    :param p_ml:
    :return:
    """
    return np.sum(np.multiply(p, np.log(np.divide(p, p_ml))))

In [2]:
def create_set(
    p_ml,
    roh,
    step,
    min_grid_p_0,
    min_grid_p_1,
    max_grid_p_0,
    max_grid_p_1,
    set_method="Kullback",
):
    """
    This function evaluates points in a given grid and returns their coordinate if
    they are in the given set.
    :param p_ml: Middle point of set
    :param roh: set size
    :param step: stepsize for grid
    :param min_grid_p_0: minimal value of p_0 in the grid
    :param min_grid_p_1: minimal value of p_1 in the grid
    :param max_grid_p_0: maximal value of p_0 in the grid
    :param max_grid_p_1: maximal value of p_1 in the grid
    :param set_method: Method to calculate distance between probability vectors
    :return:
    """
    if set_method == "Kullback":
        set_cond = Kullback_Leibler
    else:
        raise ValueError("Set measure not supported.")
    set_coordinates = np.empty(shape=(1, 2))
    for p_0 in np.arange(min_grid_p_0, max_grid_p_0, step):
        for p_1 in np.arange(min_grid_p_1, max_grid_p_1, step):
            p = np.array([p_0, p_1, 1 - p_0 - p_1])
            if p[(p > 0) & (p < 1)].size == p.size:
                if roh - Kullback_Leibler(p, p_ml) >= 0:
                    set_coordinates = np.append(set_coordinates, [[p_0, p_1]], axis=0)
    return set_coordinates[1:]

In [3]:
import scipy as sc
p_ml_0 = 0.4
p_ml_1 = 0.25
p_ml_2 = 1 - p_ml_0 - p_ml_1
p_ml = np.array([p_ml_0, p_ml_1, p_ml_2])
rho = 0.06
min_grid_p_0 = min_grid_p_1 = 0
max_grid_p_0 = max_grid_p_1 = 1
step = 0.01
set_probs_pre = create_set(
    p_ml,
    rho,
    step,
    min_grid_p_0,
    min_grid_p_1,
    max_grid_p_0,
    max_grid_p_1,
)
min_grid_p_0 = set_probs_pre[:, 0].min() - step
max_grid_p_0 = set_probs_pre[:, 0].max() + step
min_grid_p_1 = set_probs_pre[:, 1].min() - step
max_grid_p_1 = set_probs_pre[:, 1].max() + step
step = 0.001
set_probs = create_set(
    p_ml,
    rho,
    step,
    min_grid_p_0,
    min_grid_p_1,
    max_grid_p_0,
    max_grid_p_1,
)
hull = sc.spatial.ConvexHull(set_probs)

AttributeError: module 'scipy' has no attribute 'spatial'

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
ax = Axes3D(fig)
verts = [[0, 1, 0], [0, 0, 1], [1, 0, 0]]
ax.set_xlim3d([1, 0])
ax.set_xlabel(r"Increase by 0", fontsize=14)
ax.set_ylim3d([1, 0])
ax.set_ylabel(r"Increase by 1", fontsize=14)
ax.set_zlim3d([0, 1])
ax.set_zlabel(r"Increase by 2", fontsize=14)

ax.scatter(verts[0], verts[1], zs=verts[2], s=100, label="True Position")
for simplex in hull.simplices:
    ax.plot3D(
        set_probs[simplex, 0],
        set_probs[simplex, 1],
        (1 - set_probs[simplex, 0] - set_probs[simplex, 1]),
        color="red",
    )
ax.plot3D([p_ml[0]], [p_ml[1]], [p_ml[2]], marker="o", markersize=4, color="red")
ax.add_collection3d(Poly3DCollection([verts], facecolor="green", alpha=0.8))
plt.savefig("../figures/probability_simplex.png")
plt.show()